In [1]:
from snowflake.snowpark import Session
# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')
import configparser

import pandas as pd

In [2]:
df =  pd.read_csv('/data/Lumin_Dump_CreditRiskAnalytics_v1.csv')

In [3]:
df.shape

(356255, 108)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356255 entries, 0 to 356254
Columns: 108 entries, RECORD_COUNT to Credit Bureau Requests
dtypes: float64(71), int64(17), object(20)
memory usage: 293.5+ MB


In [6]:
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

['snowflake_connection.ini']

In [7]:
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    #"password": os.getenv('snowflake_password'),
    "account": f'{config["Snowflake"]["account"]}',
    #"account": os.getenv('snowflake_account'),
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

In [8]:
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [13]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df.select_dtypes(include=numerics).columns

Index(['RECORD_COUNT', 'SK_ID_CURR', 'PD', 'TARGET', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'AGE', 'DAYS_EMPLOYED',
       'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL',
       'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT_W_CITY',
       'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION',
       'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
       'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY',
       'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG',
       'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
       'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
       'NONLIVINGAPARTM

In [21]:
df['HOUSETYPE_MODE'].fillna('Missing', inplace=True)
df['WALLSMATERIAL_MODE'].fillna('Missing', inplace=True)
df['EMERGENCYSTATE_MODE'].fillna('Missing', inplace=True)

In [22]:
df_snowflake=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_snowflake.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CRA_APPLICATION_OUTPUT_DECISION")

TypeError: field OCCUPATION_TYPE: Cannot merge type <class 'snowflake.snowpark.types.StringType'> and <class 'snowflake.snowpark.types.FloatType'>